In [5]:
from google.cloud import vision
from google.oauth2 import service_account
from google.cloud import vision_v1
from google.protobuf import json_format
import os
import re

key_path = 'KEY_FILE.json'
credentials = service_account.Credentials.from_service_account_file(key_path)
client = vision.ImageAnnotatorClient(credentials=credentials)

def get_values_from_image(image_file_name):

    # Read the image file
    with open(image_file_name, 'rb') as image_file:
        content = image_file.read()

    # Create an image object
    image = vision_v1.Image(content=content)
    image_context = vision_v1.ImageContext(language_hints=["en"])

    # Perform text detection on the image
    response = client.text_detection(image=image, image_context=image_context)
    extracted_text = response.text_annotations

    cleaned_text = ""
    for text in extracted_text:
        text_description = text.description
        if isinstance(text_description, str):
            cleaned_text += re.sub(r'[^\w\s./:]', '', text_description) + " "

    one_line_text = " ".join(cleaned_text.split())

    # Initialize the spaCy NER model (assuming 'nlp_ner' is a valid NER model)
    nlp_ner = spacy.load("model-best")

    # Process the one-line text with spaCy NER
    doc = nlp_ner(one_line_text)

    # Extract entities
    readings = [ent.text for ent in doc.ents if ent.label_ == "BG READING"]
    units = [ent.text for ent in doc.ents if ent.label_ == "UNITS"]
    date = [ent.text for ent in doc.ents if ent.label_ == "DATE"]
    time = [ent.text for ent in doc.ents if ent.label_ == "TIME"]
    deviceName = [ent.text for ent in doc.ents if ent.label_ == "DEVICE NAME"]

    # Create a dictionary to store the results
    result = {
        "Reading": readings,
        "units": units,
        "date": date,
        "time": time,
        "deviceName": deviceName
    }

    return result

image_file_name = 'bgReading_4.jpeg'
result = get_values_from_image(image_file_name)
print(result)

{'Reading': ['47'], 'units': ['mg/dL'], 'date': ['10/26/20'], 'time': ['9:25 am'], 'deviceName': ['ACCUCHEK Guide']}
